<a href="https://colab.research.google.com/github/alelorys/chatbot-flask/blob/main/chatBot_flask_testowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/alelorys/chatbot-flask

Cloning into 'chatbot-flask'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 39 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [2]:
!pip install flask-ngrok
!pip install pystempel

!pip install tflearn
!pip install pyspellchecker 

     |████████████████████████████████| 2.7 MB 5.1 MB/s 
     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 3.3 MB 50.7 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 107 kB 5.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=619d87633fb081febd13868c23e9c26cd043e88d8a33e3d69a0b92542e4930d5
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
     |████████████████████████████████| 2.7 MB 5.0 MB/s 


In [3]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import json
import pickle
import random
import difflib

import nltk
import numpy
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_yaml
from stempel import StempelStemmer

#/content/drive/MyDrive/Colabooks/chatBoTest/static
#/content/drive/MyDrive/Colabooks/chatBoTest/templates
TEMPLATE = '/content/chatbot-flask/templates'
STATIC = '/content/chatbot-flask/static'
nltk.download('punkt')

app = Flask(__name__, template_folder=TEMPLATE, static_folder=STATIC)
run_with_ngrok(app)

stemmer = StempelStemmer.polimorf()

with open("/content/chatbot-flask/q&a.json") as file:
    data = json.load(file)

try:
    with open("/content/chatbot-flask/chatbot.pickle", "rb") as file:
        words, labels, training, output = pickle.load(file)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["questionAnswer"]:
        for pattern in intent["patterns"]:
            wrds = pattern.split(" ")
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("/content/chatbot-flask/chatbot.pickle", "wb") as file:
        pickle.dump((words, labels, training, output), file)

try:
    yaml_file = open('/content/chatbot-flask/chatbotmodel.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    myChatModel = model_from_yaml(loaded_model_yaml)
    myChatModel.load_weights("/content/chatbot-flask/chatbotmodel.h5")
    

except:
    # Make our neural network
    myChatModel = Sequential()
    myChatModel.add(Dense(16, input_shape=[len(words)], activation='relu'))
    myChatModel.add(Dense(16, activation='relu'))
    myChatModel.add(Dense(len(labels), activation='softmax'))

    # optimize the model
    myChatModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # train the model
    myChatModel.fit(training, output, epochs=160, batch_size=8, shuffle=True)

    # serialize model to json and save it to disk
    model_json = myChatModel.to_json()
    with open("/content/chatbot-flask/chatbotmodel.json", "w") as j_file:
        j_file.write(model_json)

    # serialize weights to HDF5
    myChatModel.save_weights("/content/chatbot-flask/chatbotmodel.h5")
    print("Saved model from disk")
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = s.split(" ")

    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
          seq = difflib.SequenceMatcher(None,w,se)
          d = seq.ratio()*100
          if w == se:
            bag[i] = 1
          if d >= 75:
            bag[i] = 1
          
         
            
    return numpy.array(bag)

def chatWithBot(userMess):
    currentText = bag_of_words(userMess, words)
    currentTextArray = [currentText]
    numpyCurrentText = numpy.array(currentTextArray)
    if numpy.all((numpyCurrentText == 0)):
      answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
      return answer

    result = myChatModel.predict(numpyCurrentText[0:1])
    result_index = numpy.argmax(result)
    tag = labels[result_index]
    if result[0][result_index] > 0.6:
        for tg in data["questionAnswer"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        answer = random.choice(responses)
        return answer

    else:
        answer = "Nie rozumiem, zadaj pytanie jeszcze raz!"
        return answer

def chat(userMess):
    

    while True:
        inp = userMess
        if inp.lower() == "quit":
            break
        if inp.lower() == "" or inp.lower() == " ":
          answer = "Musisz coś napisać!"
          return answer
        
        return chatWithBot(inp)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userMess = request.args.get("msg")
    return chat(userMess)


if __name__ == "__main__":
    app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loading: 100%|██████████| 11368252/11368252 [00:14<00:00, 807366.36bytes/s]


Epoch 1/160
26/26 [==============================] - 1s 2ms/step - loss: 4.4442 - accuracy: 0.0149 
Epoch 2/160
26/26 [==============================] - 0s 1ms/step - loss: 4.4321 - accuracy: 0.0446
Epoch 3/160
26/26 [==============================] - 0s 1ms/step - loss: 4.4213 - accuracy: 0.0644
Epoch 4/160
26/26 [==============================] - 0s 1ms/step - loss: 4.4076 - accuracy: 0.0693
Epoch 5/160
26/26 [==============================] - 0s 2ms/step - loss: 4.3891 - accuracy: 0.0891
Epoch 6/160
26/26 [==============================] - 0s 2ms/step - loss: 4.3647 - accuracy: 0.1040
Epoch 7/160
26/26 [==============================] - 0s 2ms/step - loss: 4.3328 - accuracy: 0.1089
Epoch 8/160
26/26 [==============================] - 0s 2ms/step - loss: 4.2895 - accuracy: 0.1238
Epoch 9/160
26/26 [==============================] - 0s 2ms/step - loss: 4.2370 - accuracy: 0.1238
Epoch 10/160
26/26 [==============================] - 0s 2ms/step - loss: 4.1720 - accuracy: 0.1485
Epoch 11

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://196f-104-196-208-225.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Oct/2021 08:01:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:01:08] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:01:08] "GET /static/chat.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:01:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Oct/2021 08:01:15] "GET /get?msg=Hejo HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:01:22] "GET /get?msg=Hejka HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:01:26] "GET /get?msg=Hej HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:01:51] "GET /get?msg=Jak%20uczyć%20się%20programowania%3F HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:02:18] "GET /get?msg=Typy%20juniorów HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 08:02:36] "GET /get?msg=O%20co%20chodzi%20z%20home%20alone%3F HTTP/1.1" 200 -
